In [16]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.mode.chained_assignment = None
from numpy import loadtxt
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
%matplotlib inline
import matplotlib
#matplotlib.use('agg')
matplotlib.style.use('ggplot')
import pickle as pkl
from matplotlib import pyplot as plt
from collections import Counter
from functools import reduce
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import random
random.seed(1991)
import glob

In [5]:
dfm_dec=pd.read_csv("/data/dharp/compounding/datasets/dist_sem_dec.csv")
dfm_dec

,modifier,head,1800_local_mi,1810_local_mi,1820_local_mi,1830_local_mi,1840_local_mi,1850_local_mi,1860_local_mi,1870_local_mi,...,1900_sim_with_modifier,1910_sim_with_modifier,1920_sim_with_modifier,1930_sim_with_modifier,1940_sim_with_modifier,1950_sim_with_modifier,1960_sim_with_modifier,1970_sim_with_modifier,1980_sim_with_modifier,1990_sim_with_modifier
0,a_n,aaaaa_n,0.006,0.005,0.004,0.003,0.003,0.002,0.002,0.001,...,0.000,0.000,0.000,0.000,0.000,0.000,0.059,0.054,0.053,0.048
1,a_n,addison_n,0.006,0.005,0.004,0.003,0.003,0.002,0.002,0.001,...,0.000,0.000,0.000,0.000,0.000,0.000,0.049,0.054,0.057,0.051
2,a_n,anti_n,0.006,0.005,0.004,0.003,0.003,0.002,0.002,0.001,...,0.000,0.000,0.000,0.000,0.000,0.000,0.072,0.063,0.059,0.055
3,a_n,archive_n,0.006,0.005,0.004,0.003,0.003,0.002,0.002,0.001,...,0.000,0.000,0.000,0.000,0.000,0.000,0.102,0.103,0.090,0.073
4,a_n,atom_n,0.006,0.005,0.004,0.003,0.003,0.002,0.002,0.001,...,0.000,0.000,0.000,0.000,0.037,0.067,0.064,0.052,0.052,0.051
5,a_n,b_n,0.007,0.008,0.006,0.006,0.007,0.008,0.011,0.012,...,0.467,0.385,0.371,0.388,0.345,0.303,0.257,0.217,0.188,0.190
6,a_n,band_n,0.006,0.005,0.004,0.003,0.003,0.002,0.002,0.001,...,0.000,0.000,0.000,0.000,0.000,0.000,0.069,0.107,0.099,0.102
7,a_n,baseball_n,0.006,0.005,0.004,0.003,0.003,0.002,0.002,0.001,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.053,0.067
8,a_n,behavior_n,0.006,0.005,0.004,0.003,0.003,0.002,0.002,0.001,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.036,0.038,0.028
9,a_n,bell_n,0.006,0.005,0.004,0.003,0.003,0.002,0.002,0.001,...,0.000,0.000,0.000,0.000,0.000,0.000,0.105,0.111,0.119,0.099


In [6]:
novel_compounds_list = pkl.load( open( "/data/dharp/compounding/datasets/novel_compounds_list.pkl", "rb" ) )
modifiers, heads = zip(*novel_compounds_list)
heads_list=list(set(heads))
modifiers_list=list(set(modifiers))

In [7]:
novel_compounds=pd.DataFrame(novel_compounds_list)
novel_compounds.columns=['modifier','head']
novel_compounds

,modifier,head
0,a_n,peaceful_n
1,a_n,robinson_n
2,a_n,supply_n
3,india_n,peaceful_n
4,india_n,standard_n
5,india_n,award_n
6,india_n,joint_n
7,india_n,companion_n
8,india_n,foundation_n
9,india_n,gentleman_n


In [8]:
modifiers=dfm_dec.loc[dfm_dec.modifier.isin(modifiers_list)].drop(['head','1800_sim_with_head','1810_sim_with_head',
'1820_sim_with_head','1830_sim_with_head','1840_sim_with_head','1850_sim_with_head','1860_sim_with_head','1870_sim_with_head',
 '1880_sim_with_head','1890_sim_with_head','1900_sim_with_head','1910_sim_with_head','1920_sim_with_head','1930_sim_with_head','1940_sim_with_head',
 '1950_sim_with_head','1960_sim_with_head','1970_sim_with_head','1980_sim_with_head','1990_sim_with_head'],axis=1)
modifier_features=modifiers.groupby(['modifier']).agg({'mean','std'})
modifier_features_1=modifier_features.columns.get_level_values(0)
modifier_features_2=modifier_features.columns.get_level_values(1)

cur_year=0
new_columns=[]
for year in modifier_features_1:
    new_columns.append("modifier_"+str(year)+"_"+modifier_features_2[cur_year])
    cur_year+=1
modifier_features.columns=new_columns
modifier_features.fillna(modifier_features.mean(),inplace=True)
modifier_features.reset_index(inplace=True)
modifier_features

,modifier,modifier_1800_local_mi_std,modifier_1800_local_mi_mean,modifier_1810_local_mi_std,modifier_1810_local_mi_mean,modifier_1820_local_mi_std,modifier_1820_local_mi_mean,modifier_1830_local_mi_std,modifier_1830_local_mi_mean,modifier_1840_local_mi_std,...,modifier_1950_sim_with_modifier_std,modifier_1950_sim_with_modifier_mean,modifier_1960_sim_with_modifier_std,modifier_1960_sim_with_modifier_mean,modifier_1970_sim_with_modifier_std,modifier_1970_sim_with_modifier_mean,modifier_1980_sim_with_modifier_std,modifier_1980_sim_with_modifier_mean,modifier_1990_sim_with_modifier_std,modifier_1990_sim_with_modifier_mean
0,a_n,0.001,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.000,...,0.058,0.033,0.054,0.045,0.050,0.049,0.048,0.051,0.043,0.056
1,aaa_n,0.000,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.000,...,0.233,0.100,0.197,0.111,0.189,0.117,0.157,0.163,0.110,0.199
2,aarhus_n,0.000,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.018,0.269
3,abbreviated_n,0.000,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.000,...,0.408,0.167,0.408,0.167,0.334,0.212,0.209,0.400,0.087,0.392
4,abduction_n,0.000,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.277,0.124,0.130,0.393
5,abecedarian_n,0.000,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.000,...,0.129,0.000,0.145,0.000,0.154,0.000,0.155,0.000,0.130,0.550
6,abilities_n,0.002,0.006,0.001,0.005,0.001,0.004,0.000,0.003,0.000,...,0.204,0.150,0.181,0.173,0.193,0.163,0.186,0.182,0.156,0.202
7,ability_n,0.000,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.000,...,0.115,0.060,0.114,0.069,0.111,0.075,0.102,0.095,0.092,0.107
8,abnormalities_n,0.000,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.000,...,0.129,0.000,0.145,0.000,0.154,0.000,0.155,0.869,0.130,0.743
9,abolition_n,0.000,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.001,...,0.104,0.065,0.099,0.089,0.108,0.072,0.106,0.073,0.103,0.085


In [9]:
heads=dfm_dec.loc[dfm_dec['head'].isin(heads_list)].drop(['modifier','1800_sim_with_modifier', '1810_sim_with_modifier',
 '1820_sim_with_modifier','1830_sim_with_modifier','1840_sim_with_modifier','1850_sim_with_modifier','1860_sim_with_modifier',
 '1870_sim_with_modifier','1880_sim_with_modifier','1890_sim_with_modifier','1900_sim_with_modifier','1910_sim_with_modifier',
 '1920_sim_with_modifier','1930_sim_with_modifier','1940_sim_with_modifier','1950_sim_with_modifier','1960_sim_with_modifier',
 '1970_sim_with_modifier','1980_sim_with_modifier','1990_sim_with_modifier'],axis=1)
head_features=heads.groupby(['head']).agg({'mean','std'})
head_features_1=head_features.columns.get_level_values(0)
head_features_2=head_features.columns.get_level_values(1)

cur_year=0
new_columns=[]
for year in head_features_1:
    new_columns.append("head_"+str(year)+"_"+head_features_2[cur_year])
    cur_year+=1
head_features.columns=new_columns
head_features.fillna(head_features.mean(),inplace=True)
head_features.reset_index(inplace=True)
head_features

,head,head_1800_local_mi_std,head_1800_local_mi_mean,head_1810_local_mi_std,head_1810_local_mi_mean,head_1820_local_mi_std,head_1820_local_mi_mean,head_1830_local_mi_std,head_1830_local_mi_mean,head_1840_local_mi_std,...,head_1950_sim_with_head_std,head_1950_sim_with_head_mean,head_1960_sim_with_head_std,head_1960_sim_with_head_mean,head_1970_sim_with_head_std,head_1970_sim_with_head_mean,head_1980_sim_with_head_std,head_1980_sim_with_head_mean,head_1990_sim_with_head_std,head_1990_sim_with_head_mean
0,a_n,0.001,0.006,0.001,0.005,0.000,0.004,0.000,0.003,0.000,...,0.046,0.029,0.044,0.036,0.043,0.038,0.040,0.039,0.037,0.042
1,aaron_n,0.000,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.000,...,0.186,0.188,0.173,0.183,0.161,0.153,0.163,0.154,0.153,0.158
2,ab_n,0.001,0.006,0.004,0.006,0.003,0.004,0.002,0.004,0.001,...,0.119,0.112,0.115,0.123,0.119,0.108,0.118,0.097,0.118,0.099
3,abandonment_n,0.000,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.000,...,0.275,0.092,0.286,0.095,0.244,0.174,0.143,0.265,0.097,0.289
4,abbreviation_n,0.000,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.000,...,0.135,0.027,0.151,0.086,0.150,0.120,0.131,0.188,0.111,0.199
5,abc_n,0.000,0.006,0.001,0.005,0.000,0.004,0.000,0.003,0.000,...,0.082,0.071,0.080,0.084,0.082,0.073,0.091,0.058,0.089,0.081
6,abduction_n,0.000,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.000,...,0.000,0.000,0.274,0.156,0.284,0.107,0.272,0.274,0.112,0.356
7,abdul_n,0.000,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.000,...,0.198,0.128,0.138,0.167,0.163,0.144,0.144,0.165,0.142,0.163
8,abdullah_n,0.000,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.000,...,0.357,0.257,0.327,0.304,0.322,0.280,0.302,0.273,0.276,0.306
9,ability_n,0.000,0.006,0.000,0.005,0.000,0.004,0.000,0.003,0.000,...,0.086,0.048,0.083,0.052,0.080,0.060,0.074,0.070,0.065,0.078


## Positive Set

In [10]:
positive_df=pd.merge(novel_compounds,head_features,on=["head"])
positive_df=pd.merge(positive_df,modifier_features,on=["modifier"])
positive_df['Plausibility']=True
positive_df

,modifier,head,head_1800_local_mi_std,head_1800_local_mi_mean,head_1810_local_mi_std,head_1810_local_mi_mean,head_1820_local_mi_std,head_1820_local_mi_mean,head_1830_local_mi_std,head_1830_local_mi_mean,...,modifier_1950_sim_with_modifier_mean,modifier_1960_sim_with_modifier_std,modifier_1960_sim_with_modifier_mean,modifier_1970_sim_with_modifier_std,modifier_1970_sim_with_modifier_mean,modifier_1980_sim_with_modifier_std,modifier_1980_sim_with_modifier_mean,modifier_1990_sim_with_modifier_std,modifier_1990_sim_with_modifier_mean,Plausibility
0,a_n,peaceful_n,0.000,0.006,0.000,0.005,0.000,0.004,0.000,0.003,...,0.033,0.054,0.045,0.050,0.049,0.048,0.051,0.043,0.056,True
1,a_n,robinson_n,0.000,0.006,0.000,0.005,0.000,0.004,0.000,0.003,...,0.033,0.054,0.045,0.050,0.049,0.048,0.051,0.043,0.056,True
2,a_n,supply_n,0.000,0.006,0.000,0.005,0.000,0.004,0.000,0.003,...,0.033,0.054,0.045,0.050,0.049,0.048,0.051,0.043,0.056,True
3,india_n,peaceful_n,0.000,0.006,0.000,0.005,0.000,0.004,0.000,0.003,...,0.048,0.054,0.060,0.053,0.062,0.051,0.060,0.050,0.060,True
4,india_n,standard_n,0.000,0.006,0.000,0.005,0.000,0.004,0.000,0.003,...,0.048,0.054,0.060,0.053,0.062,0.051,0.060,0.050,0.060,True
5,india_n,award_n,0.000,0.006,0.000,0.005,0.000,0.004,0.000,0.003,...,0.048,0.054,0.060,0.053,0.062,0.051,0.060,0.050,0.060,True
6,india_n,joint_n,0.001,0.006,0.001,0.005,0.001,0.004,0.001,0.004,...,0.048,0.054,0.060,0.053,0.062,0.051,0.060,0.050,0.060,True
7,india_n,companion_n,0.001,0.006,0.001,0.005,0.001,0.004,0.001,0.004,...,0.048,0.054,0.060,0.053,0.062,0.051,0.060,0.050,0.060,True
8,india_n,foundation_n,0.000,0.006,0.000,0.005,0.000,0.004,0.000,0.003,...,0.048,0.054,0.060,0.053,0.062,0.051,0.060,0.050,0.060,True
9,india_n,gentleman_n,0.010,0.007,0.008,0.006,0.007,0.005,0.006,0.004,...,0.048,0.054,0.060,0.053,0.062,0.051,0.060,0.050,0.060,True


## Negative Sets

In [11]:
def neg_df_creator(file):
    pkl_file=pkl.load( open(file,'rb'))
    df=pd.DataFrame(pkl_file)
    df.columns=['modifier','head']
    negative_df=pd.merge(df,head_features,on=["head"])
    negative_df=pd.merge(negative_df,modifier_features,on=["modifier"])
    negative_df['Plausibility']=False
    return negative_df

In [12]:
def df_joiner(files):
    df_list=[]
    for file in files:
        neg_df=neg_df_creator(file)
        whole_df=pd.concat([neg_df,positive_df])
        df_list.append(whole_df)
    
    return df_list

In [13]:
corrupt_modifier_files=[]
for file in glob.glob("/data/dharp/compounding/datasets/corrupt_modifier*"):
    corrupt_modifier_files.append(file)
corrupt_modifiers=df_joiner(corrupt_modifier_files)

In [14]:
corrupt_head_files=[]
for file in glob.glob("/data/dharp/compounding/datasets/corrupt_head*"):
    corrupt_head_files.append(file)
corrupt_heads=df_joiner(corrupt_head_files)

## XGBoost

In [19]:
acc_ch=[]
for i,corrupt_head in enumerate(corrupt_heads):
    data=corrupt_head.drop(['modifier','head'],axis=1)
    X, Y = data.iloc[:,:-1],data.iloc[:,-1]
    
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1991)
    # fit model no training data
    model = XGBClassifier(njobs=-1)
    model.fit(X_train, y_train)
    # make predictions for test data
    y_pred = model.predict(X_test)
    predictions = [round(value) for value in y_pred]
    # evaluate predictions
    print("DF",i+1)
    accuracy = accuracy_score(y_test, predictions)
    acc_ch.append(accuracy)
    print("Accuracy: %.2f%%" % (accuracy * 100.0))


DF 1
Accuracy: 70.92%
DF 2
Accuracy: 71.34%
DF 3
Accuracy: 71.28%
DF 4
Accuracy: 71.08%
DF 5
Accuracy: 71.59%
DF 6
Accuracy: 70.91%
DF 7
Accuracy: 71.38%
DF 8
Accuracy: 71.38%
DF 9
Accuracy: 71.60%
DF 10
Accuracy: 71.47%


In [20]:
np.mean(acc_ch)

0.7129525483304042

In [21]:
np.std(acc_ch)

0.002372973996371425

In [22]:
acc_cm=[]
for i,corrupt_head in enumerate(corrupt_modifiers):
    data=corrupt_head.drop(['modifier','head'],axis=1)
    X, Y = data.iloc[:,:-1],data.iloc[:,-1]
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1991)
# fit model no training data
    model = XGBClassifier(njobs=-1)
    model.fit(X_train, y_train)
# make predictions for test data
    y_pred = model.predict(X_test)
    predictions = [round(value) for value in y_pred]
# evaluate predictions
    accuracy = accuracy_score(y_test, predictions)
    acc_cm.append(accuracy)
    print("DF",i+1)
    print("Accuracy: %.2f%%" % (accuracy * 100.0))

DF 1
Accuracy: 68.21%
DF 2
Accuracy: 68.24%
DF 3
Accuracy: 68.67%
DF 4
Accuracy: 68.81%
DF 5
Accuracy: 68.45%
DF 6
Accuracy: 67.50%
DF 7
Accuracy: 68.13%
DF 8
Accuracy: 68.60%
DF 9
Accuracy: 68.42%
DF 10
Accuracy: 68.24%


In [23]:
np.mean(acc_cm)

0.6832776801405976

In [24]:
np.std(acc_cm)

0.0034876024015021165